In [1]:
import random
import numpy as np
import pandas as pd
from random import sample

In [7]:
class City: # gene
#     __distances_table = {}
    def __init__(self, location):
        self.name = location[0]
        self.x_value = location[1]
        self.y_value = location[2]
        
    def get_distance_to(self, dest):
        x_origin, y_origin = self.x_value, self.y_value
        x_dest, y_dest = dest.x_value, dest.y_value
        return ((x_dest-x_origin)**2 + (y_dest-y_origin)**2)**0.5
    
    def get_city(self):
        return (self.x_value, self.y_value)
    
class Individual:
    def __init__(self, citys):
        self.citys = citys
        self.__cost = 0
        
        self.skill_facter = 0
        self.scalar_fitness = 0
    
    @property
    def cost(self):
        if(self.__cost == 0):
            for i in range(len(self.citys)):
                self.__cost += self.citys[i].get_distance_to(self.citys[i-1])
        return self.__cost
    
    def get_individual(self):
        return [city.get_city() for city in self.citys]
    
    def get_name(self):
        return [city.name for city in self.citys]

# pop
class Population:
    def __init__(self, individuals):
        self.individuals = individuals
        self.__length = len(individuals)
        
    @property
    def length(self):
        self.__length = len(self.individuals)
        return self.__length
    
    def add(self, indivs):
        for indiv in indivs:
            self.individuals.append(indiv)
    
    def add_nums(self, nums):
        citys = self.individuals[0].citys.copy()
        individuals = []
        for _ in range(nums):
            np.random.shuffle(citys)
            individuals.append(Individual(citys.copy()))
        
        self.add(individuals)
    def remove(self, indivs):
        for indiv in indivs:
            self.individuals.remove(indiv)
    
    def get_fittest(self):
        best_indiv = self.individuals[0]
        
        best_indiv1, best_indiv2 = decryption(self.individuals[0], citys_1, citys_2)
        for indiv in self.individuals:
            indiv1, indiv2 = decryption(indiv, citys_1, citys_2)
            
            if(indiv1.cost < best_indiv1.cost):
                best_indiv1 = indiv1
            if(indiv2.cost < best_indiv2.cost):
                best_indiv2 = indiv2
        
        return [best_indiv1, best_indiv2]

def decryption(indiv, citys_1, citys_2):
    index1 = [indiv.citys.index(city) for city in citys_1]
    index2 = [indiv.citys.index(city) for city in citys_2]
    
    index1.sort()
    index2.sort()
    return [Individual([indiv.citys[index] for index in index1]), 
            Individual([indiv.citys[index] for index in index2])]

def reset_param(pop):
    # rank
    cost1 = []
    cost2 = []
    for indiv in pop.individuals:
        x1, x2 = decryption(indiv, citys_1, citys_2)
        cost1.append(x1.cost)
        cost2.append(x2.cost)
        
    cost1.sort()
    cost2.sort()
    
    for indiv in pop.individuals:
        x1, x2 = decryption(indiv, citys_1, citys_2)
        if(cost1.index(x1.cost) < cost2.index(x2.cost)):
            indiv.skill_facter = 1
            indiv.scalar_fitness = 1/(cost1.index(x1.cost)+1)
        else:
            indiv.skill_facter = 2
            indiv.scalar_fitness = 1/(cost2.index(x2.cost)+1)

def crossover(parent1, parent2, rmp):
    if( (parent1.skill_facter == parent2.skill_facter) or (random.random()< rmp) ): 
        len_gene = len(parent1.citys)
        t1 = len_gene//3-1
        t2 = len_gene*2//3

        child1 = parent1.citys.copy()
        child2 = parent2.citys.copy()

        child1[t1:t2] = parent2.citys[t1:t2]
        child2[t1:t2] = parent1.citys[t1:t2]
        for i in range(t1):
            while(child1[i] in child1[t1:t2]):
                child1[i] = parent1.citys[parent2.citys.index(child1[i])]

            while(child2[i] in child2[t1:t2]):
                child2[i] = parent2.citys[parent1.citys.index(child2[i])]

        for i in range(t2, len_gene):
            while(child1[i] in child1[t1:t2]):
                child1[i] = parent1.citys[parent2.citys.index(child1[i])]

            while(child2[i] in child2[t1:t2]):
                child2[i] = parent2.citys[parent1.citys.index(child2[i])]

        return [Individual(child1), Individual(child2)]
    else: # muatate
        mutate1 = mutate(parent1)
        mutate2 = mutate(parent2)
        return [mutate1, mutate2]

def mutate(indiv):
    citys = indiv.citys.copy()
    a, b = sample(list(range(len(citys))), 2)
        
    # swap
    temp = citys[a]
    citys[a] = citys[b]
    citys[b] = temp
    return Individual(citys)

def selection(pop, max_len):
    reset_param(pop)
    pop.individuals.sort(key = lambda x: x.scalar_fitness, reverse=True)
    pop.remove(pop.individuals[max_len:])

def evolution(pop, max_len, rmp):
    # select parent
    reset_param(pop)
    for _ in range(max_len//2):
        a, b, c, d = sample(pop.individuals, 4)
        mang1 = [a, b]
        mang2 = [c, d]
        
        mang1.sort(key= lambda x: x.scalar_fitness)
        mang2.sort(key= lambda x: x.scalar_fitness)
        
        # crossover
        pop.add(crossover(mang1[-1], mang2[-1], rmp))
    selection(pop, max_len)
    return [x.cost for x in pop.get_fittest()]

In [8]:
citys = [City(x) for x in pd.read_csv("dataset/att48.csv").values]

In [11]:
citys_1 = citys[:25]
citys_2 = citys

pop = Population([Individual(citys)])
pop.add_nums(99)

In [12]:
for _ in range(1000):
    print("generation {0} scalar_fitness {1}".format(_, evolution(pop, max_len=100, rmp=0.3)))

generation 0 scalar_fitness [56375.2617066886, 112187.50783680534]
generation 1 scalar_fitness [56375.2617066886, 112187.50783680534]
generation 2 scalar_fitness [52560.652815559646, 112187.50783680534]
generation 3 scalar_fitness [52560.652815559646, 112187.50783680534]
generation 4 scalar_fitness [49798.03278815957, 110197.53340904885]
generation 5 scalar_fitness [49172.48218551776, 104595.02483487893]
generation 6 scalar_fitness [49172.48218551776, 104595.02483487893]
generation 7 scalar_fitness [48621.24535221088, 104595.02483487893]
generation 8 scalar_fitness [48621.24535221088, 104595.02483487893]
generation 9 scalar_fitness [48621.24535221088, 104595.02483487893]
generation 10 scalar_fitness [48621.24535221088, 101627.5655058945]
generation 11 scalar_fitness [46473.266128878364, 99878.44863619689]
generation 12 scalar_fitness [45357.03832596316, 98261.07536596371]
generation 13 scalar_fitness [45357.03832596316, 96532.62077667208]
generation 14 scalar_fitness [45357.03832596316

generation 122 scalar_fitness [30802.962135409678, 67290.1652617335]
generation 123 scalar_fitness [30802.962135409678, 67290.1652617335]
generation 124 scalar_fitness [30802.962135409678, 67290.1652617335]
generation 125 scalar_fitness [30802.962135409678, 67290.1652617335]
generation 126 scalar_fitness [30802.962135409678, 67290.1652617335]
generation 127 scalar_fitness [30802.962135409678, 67290.1652617335]
generation 128 scalar_fitness [30802.962135409678, 67290.1652617335]
generation 129 scalar_fitness [30802.962135409678, 67290.1652617335]
generation 130 scalar_fitness [30802.962135409678, 67290.1652617335]
generation 131 scalar_fitness [30802.962135409678, 67290.1652617335]
generation 132 scalar_fitness [30802.962135409678, 67290.1652617335]
generation 133 scalar_fitness [30802.962135409678, 67290.1652617335]
generation 134 scalar_fitness [30802.962135409678, 67290.1652617335]
generation 135 scalar_fitness [30802.962135409678, 67290.1652617335]
generation 136 scalar_fitness [308

generation 240 scalar_fitness [28111.739823556494, 113433.01307704448]
generation 241 scalar_fitness [28111.739823556494, 140079.7229788982]
generation 242 scalar_fitness [28111.739823556494, 140079.7229788982]
generation 243 scalar_fitness [28111.739823556494, 140079.7229788982]
generation 244 scalar_fitness [28111.739823556494, 140079.7229788982]
generation 245 scalar_fitness [28111.739823556494, 140079.7229788982]
generation 246 scalar_fitness [28111.739823556494, 140079.7229788982]
generation 247 scalar_fitness [28111.739823556494, 140079.7229788982]
generation 248 scalar_fitness [28111.739823556494, 140079.7229788982]
generation 249 scalar_fitness [28111.739823556494, 140079.7229788982]
generation 250 scalar_fitness [28111.739823556494, 140079.7229788982]
generation 251 scalar_fitness [28111.739823556494, 140079.7229788982]
generation 252 scalar_fitness [28111.739823556494, 140079.7229788982]
generation 253 scalar_fitness [28111.739823556494, 140079.7229788982]
generation 254 scal

KeyboardInterrupt: 